<a href="https://colab.research.google.com/github/norrisandrew/Materials_Project/blob/main/MaterialsProject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, matplotlib.pyplot as plt, scipy as sp  
import xlrd, csv      # for read to .xls, read/write to csv
from sklearn.model_selection import train_test_split    # machine learning library
from sklearn import datasets
import sklearn.linear_model
import sklearn.neighbors

In [ ]:
!pip install pymatgen==2022.0.17 # <-------------- EXTREMELY IMPORTANT

In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Composition
import re
import pprint
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [ ]:
# Make sure that you have the Materials API key. Put the key in the call to
# MPRester if needed, e.g, MPRester("MY_API_KEY")
mpr = MPRester("iR6z9LivAfSmJ0Zdd")

In [ ]:
elastic_data = mpr.query({"elasticity": {"$exists": True}},              # all the elastic data with materials ids, see http://matgenb.materialsvirtuallab.org/2017/03/02/Getting-data-from-Materials-Project.html
                         properties=["task_id", "pretty_formula", "elasticity", "elastic_tensor","structure"])

In [ ]:
print(len(elastic_data))
pprint.pprint(elastic_data[1])
type(elastic_data)

In [ ]:
pprint.pprint(elastic_data[0]["pretty_formula"])
pprint.pprint(elastic_data[0]["elasticity"]["elastic_tensor"])

SpacegroupAnalyzer(elastic_data[2]["structure"]).get_crystal_system()   # 
#struct = (elastic_data[0]["structure"])
#finder = SpacegroupAnalyzer(elastic_data[0]["structure"])
#finder.get_crystal_system() 
#finder.get_lattice_type() 
#finder.get_space_group_number() 

# Now generate the 13k sets of elastic moduli along with the symmetry type.   This takes a few minutes

In [ ]:
r_array = []
sym_array = []
sgn_array = []
l1_array = []
l2_array = []
l3_array = []
l4_array = []
l5_array = []
l6_array = []
length_elastic_data = len(elastic_data)

for i in range(length_elastic_data):
        r_array.append(elastic_data[i]["pretty_formula"])     # the chemical formula

        findsym = SpacegroupAnalyzer(elastic_data[i]["structure"])       # THIS SLOWS IT DOWN, so it takes about 2.5 minutes
        sym_array.append( findsym.get_crystal_system() )       # ['triclinic', 'monoclinic', 'orthorhombic', 'tetragonal', 'trigonal', 'hexagonal', 'cubic']
        sgn_array.append( findsym.get_space_group_number() )   # the space group number 

        l1_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][0])
        l2_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][1])
        l3_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][2])
        l4_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][3])
        l5_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][4])
        l6_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][5])

Big_Data = {'Formula': r_array, 'Symmetry': sym_array, 'Space group': sgn_array, 'Elastic moduli row 1': l1_array, 'Row 2': l2_array, 'Row 3': l3_array, 'Row 4': l4_array, 'Row 5': l5_array, 'Row 6': l6_array}

r_array = np.array(r_array)        # now convert the lists to arrays for easier manipulation
sym_array = np.array(sym_array)
sgn_array = np.array(sgn_array)
l1_array = np.array(l1_array)
l2_array = np.array(l2_array)
l3_array = np.array(l3_array)
l4_array = np.array(l4_array)
l5_array = np.array(l5_array)
l6_array = np.array(l6_array)

In [ ]:
Big_Data = {'Formula': r_array, 'Symmetry': sym_array, 'Space group': sgn_array, 'Elastic moduli row 1': l1_array, 'Row 2': l2_array, 'Row 3': l3_array, 'Row 4': l4_array, 'Row 5': l5_array, 'Row 6': l6_array}

In [ ]:
import pandas as pd         #  Let's see what the data looks like

DATA = pd.DataFrame(Big_Data)
top = DATA.head(9)
top

In [ ]:
def find_indices(arr_to_check, item_to_find):              #  adapted from https://datagy.io/python-list-find-all-index/
    indices = np.where(arr_to_check == item_to_find)[0]
    return list(indices)

# Get the symmetry indices for  ['triclinic', 'monoclinic', 'orthorhombic', 'tetragonal', 'trigonal', 'hexagonal', 'cubic']
ind_cub  = find_indices(sym_array, "cubic")
ind_hex  = find_indices(sym_array, "hexagonal")
ind_trig = find_indices(sym_array, "trigonal")
ind_tetr = find_indices(sym_array, "tetragonal")
ind_orth = find_indices(sym_array, "orthorhombic")
ind_mono = find_indices(sym_array, "monoclinic")
ind_tric = find_indices(sym_array, "triclinic")

tric_n  = len(ind_tric)                  # for list rather than array was sym_array.count("triclinic")
mono_n  = len(ind_mono) 
orth_n  = len(ind_orth) 
tetr_n  = len(ind_tetr)  
trig_n  = len(ind_trig)  
hex_n   = len(ind_hex)  
cub_n   = len(ind_cub)  
all_n   = tric_n + mono_n + orth_n + tetr_n + trig_n + hex_n + cub_n
Sym_numbers = {'triclinic': tric_n, 'monoclinic': mono_n, 'orthorhombic': orth_n, 
               'tetragonal': tetr_n, 'trigonal': trig_n, 'hexagonal': hex_n, 'cubic': cub_n, 'all': all_n}
Sym_numbers

# Now we can get the elastic parameters for each symmetry group


In [ ]:
sgn_array_cub  = sgn_array[np.ix_(ind_cub)]      #   Space group numbers for the cubic materials, etc
sgn_array_hex  = sgn_array[np.ix_(ind_hex)]
sgn_array_trig = sgn_array[np.ix_(ind_trig)]
sgn_array_tetr = sgn_array[np.ix_(ind_tetr)]
sgn_array_orth = sgn_array[np.ix_(ind_orth)]
sgn_array_mono = sgn_array[np.ix_(ind_mono)]
sgn_array_tric = sgn_array[np.ix_(ind_tric)]
print( sgn_array_cub[1:12] )                     #   First 12 Space group numbers for the cubic materials, etc
print( sgn_array_hex[1:12] )
print( sgn_array_trig[1:12] )
print( sgn_array_tetr[1:12] )
print( sgn_array_orth[1:12] )
print( sgn_array_mono[1:12] )
print( sgn_array_tric[1:12] )
print(l1_array[2:6] )
print(ind_tric[1:7])


# Now compute the cubic elastic moduli for each of the 13k or so materials in the elastic data.

In [ ]:
#  for a 6x6 C find distances and moduli
def dist_cub_and_iso(C):
  CdotH  = C[0,0]+C[1,1]+C[2,2]
  Cdot3J = CdotH + 2*(C[0,1]+C[1,2]+C[0,2])
  CdotI  = 2*np.trace(C) - CdotH 
  kap = Cdot3J/9                 # kappa or bulk modulus
  mu1 = (CdotI - CdotH)/6        # mu1 which is the same as mu in DOI: 10.1121/1.2173525
  mu2 = CdotH/4 - Cdot3J/12      # mu2 which is the same as eta 
  mu = (3*mu1+2*mu2)/5           #  isotropic mu

  C11 = kap + mu2*4/3   # cubic C11
  C12 = kap - mu2*2/3   # cubic C12
  C66 = mu1             # cubic C66
  C_cub = np.array( [ [C11,C12,C12,0,0,0], [C12,C11,C12,0,0,0], [C12,C12,C11,0,0,0], [0,0,0,C66,0,0], [0,0,0,0,C66,0], [0,0,0,0,0,C66] ])   # the cubic approximation of C
  C11 = kap + mu*4/3    # isotropic C11
  C12 = kap - mu*2/3    # isotropic C12
  C66 = mu              # isotropic C66
  C_iso = np.array( [ [C11,C12,C12,0,0,0], [C12,C11,C12,0,0,0], [C12,C12,C11,0,0,0], [0,0,0,C66,0,0], [0,0,0,0,C66,0], [0,0,0,0,0,C66] ])   # the isotopic approximation of C

  C_mag    = np.linalg.norm(C,2)     #   norm(C,2) is the same as norm(C)
  dist_cub = np.linalg.norm(C - C_cub)/C_mag    #  the relative distance of C_cub from C, in an L2 sense
  dist_iso = np.linalg.norm(C - C_iso)/C_mag    #  the relative distance of C_iso from C, in an L2 sense

  return [dist_cub, dist_iso, kap, mu1, mu2, mu]

In [ ]:
form_list = [] # formula
sym_list  = [] # symmetry, make sure to run the block two above for this to work
k_list  = []   # kappa
m1_list = []   # shear modulus 1
m2_list = []   # shear modulus 2
m_list  = []   # shear modulus of iso
dc_list = []   # relative distance from cubic approximation
di_list = []   # relative distance from isotropic approximation

#for k in range(length_elastic_data):       # use these 2 lines for the entire 13k database
#  i = k

indx = ind_tric           #   consider the symmetry indicated with these 3 lines 
for k in range(len(indx)):
  i = np.array( indx[k] )

  C = np.array( elastic_data[i]["elasticity"]["elastic_tensor"] )      # converts list to array
  X = dist_cub_and_iso(C)                         # calls the function above
  
  dist_cub = X[0]
  if dist_cub < 1:
    form_list.append(elastic_data[i]["pretty_formula"])
    sym_list.append(sym_array[i])
    dc_list.append( X[0] )   #   dist to cub
    di_list.append( X[1] )   #   dist to iso
    k_list.append( X[2] )    #   kap for cub and iso
    m1_list.append( X[3] )   #   mu1 of cub
    m2_list.append( X[4] )   #   mu2 of cub
    m_list.append( X[5] )    #   mu of iso
                      
Cubic_data = {'Formula': form_list, 'Symmetry': sym_list, 'kappa': k_list, 'mu1': m1_list, 'mu2': m2_list,  'mu': m_list, 'rel dist to cubic': dc_list, 'rel dist to isotropic': di_list}

dc_list_sort = np.sort(dc_list)            # sort the distances
di_list_sort = np.sort(di_list)            # sort the distances
sym_list_sort = np.sort(sym_list)          # sort the syms

plt.plot( di_list_sort )
plt.ylabel('Relative distance to cubic')
plt.grid(True)
plt.show()

print(C)

print( sym_list.count("hexagonal") )


The plot shows the "distance to cubic" for all 13000 materials.  A zero distance means it is actucally a cubic crystal.  A non-zero value means the crystal structure is more complicated.  A small distance means it is "almost" cubic, i.e. its properties canbe approximated by a cubic materia with the three elastic moduli: kap, mu1 and mu2

In [ ]:
print( dc_list_sort[0:12] )
 